Reproducible steps for getting 10-k filings from [EDGAR](https://www.sec.gov/edgar).

You will need the latest copy of the modules.

1. [sec-downloader](https://github.com/Elijas/sec-downloader)
2. [sec-parser](https://github.com/alphanome-ai/sec-parser)


In [ ]:
pip install --upgrade pip

In [ ]:
!pip install sec-downloader
!pip install sec-parser

# Manual

1. Navigate to [here](https://www.sec.gov/edgar/searchedgar/companysearch)
2. Enter "ENV" into the search box.
3. Right hand side, expand "10-K (annual reports) and 10-Q (quarterly reports)"
4. Get whatever is on top.
5. Scroll to "Item 7"

As of 2024/01/03

"ENV" is:

* https://www.sec.gov/edgar/browse/?CIK=1337619
* https://www.sec.gov/ix?doc=/Archives/edgar/data/1337619/000133761923000012/env-20221231.htm
* pages 38-65

"MSFT" is:

* https://www.sec.gov/edgar/browse/?CIK=789019
* https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm
* pages 40-57

# Python

As below

In [1]:
from sec_downloader import Downloader

dl = Downloader("SokpheanalHuynh", "sokpheanal.huynh@gmail.com")
html_1: bytes = dl.get_filing_html(ticker = "ENV", form = "10-K")
html_2: bytes = dl.get_filing_html(ticker = "MSFT", form = "10-K")

In [6]:
# parser
# x1 = table > tr > td > child contains "Management's Discussion and Analysis of Financial Condition and Results of Operations"
# id1 = capture prior a (href="#...")
# x2 = table > tr > td > child contains "Quantitative and Qualitative Disclosures About Market Risk"
# id2 = capture prior a (href="#...")
# find children between id1 and id2

from lxml import etree
parser = etree.XMLParser(encoding = 'utf-8', recover = True, ns_clean = True)
root: etree.Element = etree.fromstring(html_2, parser)

 Per [this](https://stackoverflow.com/questions/38936185/etree-xpath-return-entire-html-instead-of-text), you need to include the NS in every part of the `xpath`

In [3]:
nsmap = {'x':'http://www.w3.org/1999/xhtml'}
xpath_1 = ".//x:table/x:tr/x:td//*[contains(text(),'Financial Condition and Results of Operations')]"
xpath_1_id = "ancestor-or-self::x:a[contains(@href, '#')]"

In [4]:
def get_ref_id(node: etree.Element, xpath: str, xpath_id: str, ns_map: dict[str, str]) -> str:
    t1: list[etree.Element] = node.xpath(xpath, namespaces = ns_map)
    t2: list[etree.Element] = t1[0].xpath(xpath_id, namespaces = ns_map)
    t3: str = t2[0].attrib['href']
    return t3

In [7]:
id1 = get_ref_id(root, xpath_1, xpath_1_id, nsmap)
print(id1)

#item_7_managements_discussion_analysis_f


TODO: use xpath to get next section
xpath1 > parent tr > next sibling >a [contains #]